# ESG EDA Notebook (World Bank via Hugging Face)

This notebook loads the **Global Environment–Social–Governance** dataset from Hugging Face, performs **basic EDA**, checks **missing values**, and renders a few **simple visualizations**.

**Dataset**: `Mahadih534/Global_Environment-Social-And-Governance-Data`
**File**: `ESGCSV.csv` (World Bank–style wide format)

> Tip: You can run this in Google Colab or GitHub Codespaces Jupyter.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)


## Load dataset

In [ ]:
try:
    # Directly from Hugging Face using fsspec-style URI
    df = pd.read_csv('hf://datasets/Mahadih534/Global_Environment-Social-And-Governance-Data/ESGCSV.csv')
    source_used = 'pandas hf:// URI'
except Exception as e:
    print('hf:// load failed, falling back to datasets library. Error:', e)
    try:
        from datasets import load_dataset
        ds = load_dataset('Mahadih534/Global_Environment-Social-And-Governance-Data')
        df = pd.DataFrame(ds['train'])
        source_used = 'datasets.load_dataset'
    except Exception as e2:
        raise RuntimeError('Failed to load dataset via both methods.')

print('Loaded via:', source_used)
print('Shape:', df.shape)
print('Columns:', list(df.columns)[:12], '...')
df.head()

## Tidy the data (wide → long)

In [ ]:
# Detect year columns (e.g., '1990', '1991', ...)
year_cols = [c for c in df.columns if str(c).isdigit()]
meta_cols = [c for c in df.columns if c not in year_cols]
if not year_cols:
    raise ValueError('No year columns detected; check the dataset structure.')

tidy = df.melt(id_vars=meta_cols, value_vars=year_cols,
               var_name='Year', value_name='Value')
tidy['Year'] = pd.to_numeric(tidy['Year'], errors='coerce')
tidy = tidy.dropna(subset=['Year'])

# Standardize column names where possible
rename_map = {
    'Country Name': 'Country_Name',
    'Country Code': 'Country_Code',
    'Indicator Name': 'Indicator_Name',
    'Indicator Code': 'Indicator_Code'
}
tidy = tidy.rename(columns=rename_map)

print('Tidy shape:', tidy.shape)
tidy.head()

## Missing data check

In [ ]:
# Missingness by indicator
missing_by_indicator = tidy.groupby('Indicator_Code')['Value'].apply(lambda s: s.isna().mean()).sort_values(ascending=False)
print('Proportion missing by indicator (top 15):')
print(missing_by_indicator.head(15))

# Missingness by country (latest decade to keep it small)
latest_year = tidy['Year'].max()
recent = tidy[tidy['Year'] >= latest_year - 10]
missing_by_country = recent.groupby('Country_Name')['Value'].apply(lambda s: s.isna().mean()).sort_values(ascending=False)
print('\nProportion missing by country, last decade (top 15):')
print(missing_by_country.head(15))

## Visualization 1: Single-indicator time series (example)

In [ ]:
example_indicator = 'EG.ELC.ACCS.ZS'   # Access to electricity (% of population)
example_country = 'Arab World'

sub = tidy[(tidy['Indicator_Code'] == example_indicator) & (tidy['Country_Name'] == example_country)].dropna(subset=['Value'])
plt.figure(figsize=(9,5))
plt.plot(sub['Year'], sub['Value'], marker='o')
plt.title(f'Access to Electricity (% of pop) – {example_country}')
plt.xlabel('Year')
plt.ylabel('% of population')
plt.grid(True)
plt.show()

## Visualization 2: Multi-country comparison (CO₂ per capita)

In [ ]:
indicator = 'EN.ATM.CO2E.PC'
countries = ['Arab World', 'United States', 'China']

sub = tidy[(tidy['Indicator_Code'] == indicator) & (tidy['Country_Name'].isin(countries))].dropna(subset=['Value'])
plt.figure(figsize=(10,6))
for c in countries:
    s = sub[sub['Country_Name']==c]
    plt.plot(s['Year'], s['Value'], marker='o', label=c)
plt.legend()
plt.title('CO₂ Emissions per Capita (metric tons)')
plt.xlabel('Year')
plt.ylabel('t CO₂ per capita')
plt.grid(True)
plt.show()

## Visualization 3: Distribution for latest year (CO₂ per capita)

In [ ]:
co2 = tidy[tidy['Indicator_Code']==indicator].dropna(subset=['Value'])
latest = int(co2['Year'].max())
co2_latest = co2[co2['Year']==latest]['Value']
plt.figure(figsize=(9,5))
plt.hist(co2_latest, bins=30)
plt.title(f'Distribution of CO₂ per Capita – {latest}')
plt.xlabel('t CO₂ per capita')
plt.ylabel('Count')
plt.grid(True)
plt.show()

## (Optional) Correlation across indicators for one country

In [ ]:
country = 'Arab World'
country_wide = tidy[tidy['Country_Name']==country].pivot_table(index='Year', columns='Indicator_Code', values='Value')

# Keep columns with enough data
country_wide = country_wide.dropna(axis=1, thresh=int(0.8*len(country_wide)))
corr = country_wide.corr()

plt.figure(figsize=(10,8))
plt.imshow(corr, aspect='auto')
plt.colorbar(label='Correlation')
plt.title(f'Indicator Correlations – {country}')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.tight_layout()
plt.show()

## Save a small tidy sample (optional)

In [ ]:
sample_inds = ['EN.ATM.CO2E.PC','EG.USE.COMM.FO.ZS','AG.LND.FRST.ZS',
               'SP.DYN.LE00.IN','SE.ADT.LITR.ZS','SH.STA.SMSS.ZS',
               'CC.EST','RL.EST','GE.EST']
small = tidy[tidy['Indicator_Code'].isin(sample_inds) & tidy['Country_Name'].isin(['Arab World','United States','China'])]
small = small.dropna(subset=['Value'])
small.to_csv('/mnt/data/esg_tidy_sample.csv', index=False)
print('Wrote /mnt/data/esg_tidy_sample.csv with shape', small.shape)
